# Advent of Code 2024

In [2]:
import numpy as np
import pandas as pd

import re
from collections import defaultdict, Counter

In [6]:
from enum import Enum


class Direction(Enum):
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3


def rotate_right(direction):
    return Direction((direction.value + 1) % 4)


def rotate_left(direction):
    return Direction((direction.value - 1) % 4)


def is_opposite(d1, d2):
    return rotate_right(rotate_right(d1)) == d2

#### 1.1

In [3]:
data = np.loadtxt("inputs/1.txt", dtype=int)
sorted_data = np.sort(data, axis=0)
np.abs(sorted_data[:, 0] - sorted_data[:, 1]).sum()

np.int64(2000468)

#### 1.2

In [4]:
counts = pd.concat(
    [
        pd.Series(sorted_data[:, 0]).value_counts().rename("0"),
        pd.Series(sorted_data[:, 1]).value_counts().rename("1"),
    ],
    axis=1,
).dropna()

(counts.iloc[:, 0] * counts.iloc[:, 1] * counts.index).sum()

np.float64(18567089.0)

#### 2.1

In [5]:
reports = [np.fromiter(line.split(), int) for line in open("inputs/2.txt")]

In [6]:
def is_safe(report):
    changes = np.diff(report)
    signs = np.sign(changes)
    return signs[0] != 0 and (signs == signs[0]).all() and (np.abs(changes) <= 3).all()


sum(map(is_safe, reports))

np.int64(502)

#### 2.2

In [7]:
def are_valid_changes(changes, possible_sign):
    return (np.sign(changes) == possible_sign) & (np.abs(changes) <= 3)


def is_safe_with_dampener(report):
    if report.size < 3:
        return True

    changes = np.diff(report)
    for possible_sign in [-1, 1]:
        errors = ~are_valid_changes(changes, possible_sign)
        n_errors = errors.sum()
        if n_errors > 2:
            continue
        elif n_errors == 2:
            idcs = np.nonzero(errors)[0]
            if idcs[0] + 1 != idcs[1] or not are_valid_changes(changes[idcs[0]] + changes[idcs[1]], possible_sign):
                continue
        elif n_errors == 1:
            idx = np.argmax(errors)
            if (
                idx != 0
                and idx != changes.size - 1
                and not are_valid_changes(changes[idx] + changes[idx + 1], possible_sign)
                and not are_valid_changes(changes[idx - 1] + changes[idx], possible_sign)
            ):
                continue
        return True
    return False


sum(map(is_safe_with_dampener, reports))

544

#### 3.1

In [8]:
memory = open("inputs/3.txt").read()

In [9]:
sum(int(a) * int(b) for a, b in re.findall(r"mul\(([0-9]+),([0-9]+)\)", memory))

180233229

#### 3.2

In [10]:
def calc_with_conditionals(memory):
    total_sum = 0
    enabled = True
    for a, b, do_string, dont_string in re.findall(r"mul\(([0-9]+),([0-9]+)\)|(do\(\))|(don't\(\))", memory):
        if a:
            if enabled:
                total_sum += int(a) * int(b)
        elif do_string:
            enabled = True
        elif dont_string:
            enabled = False
    return total_sum


calc_with_conditionals(memory)

95411583

#### 4.1

In [11]:
txt = np.array(list(map(list, open("inputs/4.txt").read().split())))

In [12]:
def count_xmas(text, words=("XMAS", "SAMX")):
    total_count = 0
    n, m = text.shape

    for view in [
        text,
        text.T,
        [np.diagonal(text, offset=offset) for offset in range(-n + 1, m)],
        [np.diagonal(np.flipud(text), offset=offset) for offset in range(-n + 1, m)],
    ]:
        total_count += sum("".join(line).count(word) for line in view for word in words)

    return total_count


count_xmas(txt)

2530

#### 4.2

In [13]:
def count_max(text):
    total_count = 0
    n, m = text.shape
    for i_min in range(n - 2):
        for j_min in range(m - 2):
            if text[i_min + 1, j_min + 1] == "A":
                i_max = i_min + 2
                j_max = j_min + 2
                diag_txt = text[[i_min, i_min, i_max, i_max], [j_min, j_max, j_min, j_max]]
                if diag_txt[0] != diag_txt[3] and np.count_nonzero(diag_txt == "S") == 2 and np.count_nonzero(diag_txt == "M") == 2:
                    total_count += 1
    return total_count


count_max(txt)

1921

#### 5.1

In [14]:
rules_txt, updates_txt = open("inputs/5.txt").read().split("\n\n")
rules = [(int(x), int(y)) for x, y in (rule.split("|") for rule in rules_txt.split())]
updates = [[int(x) for x in line.split(",")] for line in updates_txt.split()]
succeeding_pages = defaultdict(set)
for rule in rules:
    succeeding_pages[rule[0]].add(rule[1])

In [15]:
from itertools import combinations


def is_correctly_ordered(update):
    return not any(x in succeeding_pages[y] for x, y in combinations(update, 2))


def middle_page(update):
    return update[(len(update) - 1) // 2]


total_sum = sum(middle_page(update) for update in updates if is_correctly_ordered(update))
print(total_sum)

6384


#### 5.2

In [16]:
from copy import copy


def fix_update(update):
    update = copy(update)
    for i, _ in enumerate(update):
        j = i + 1
        while j < len(update):
            if update[i] in succeeding_pages[update[j]]:
                update[i], update[j] = update[j], update[i]
                j = i + 1
            else:
                j += 1
    return update


incorrect_updates = (update for update in updates if not is_correctly_ordered(update))
sum(middle_page(fix_update(update)) for update in incorrect_updates)


5353

#### 6.1

In [17]:
dx_vectors = np.array(
    [
        [-1, 0],
        [0, 1],
        [1, 0],
        [0, -1],
    ]
)


def get_dx(direction):
    return dx_vectors[direction.value]


def get_direction(symbol):
    return Direction("^>V<".index(symbol))


class Grid:
    def __init__(self, grid_txt):
        self.grid = np.array(list(map(list, grid_txt.split())))
        self.n, self.m = self.grid.shape
        self.reset_history()

    def reset_history(self):
        self.history = np.zeros((*self.grid.shape, 4), bool)

    def get(self, pos):
        return self.grid[tuple(pos)]

    def move_forward(self, pos, direction):
        return pos + get_dx(direction)

    def next_state(self, pos, direction):
        new_pos = self.move_forward(pos, direction)
        if self.out_of_bounds(new_pos):
            return new_pos, direction, True
        if self.get(new_pos) == "#":
            direction = rotate_right(direction)
        else:
            pos = new_pos
        return pos, direction, False

    def find_guard_position(self):
        for i in range(self.n):
            for j in range(self.m):
                if self.grid[i, j] != "." and self.grid[i, j] != "#":
                    return i, j
        return -1, -1

    def out_of_bounds(self, pos):
        return pos[0] < 0 or pos[0] >= self.n or pos[1] < 0 or pos[1] >= self.m

    def mark_visited(self, pos, direction):
        self.history[pos[0], pos[1], direction.value] = True

    def already_visited(self, pos, direction):
        return self.history[pos[0], pos[1], direction.value]

    def count_visited(self):
        return np.count_nonzero(self.history.any(axis=2))


def distinct_positions(grid_txt):
    grid = Grid(grid_txt)
    pos = np.array(grid.find_guard_position())
    direction = get_direction(grid.get(pos))
    out_of_bounds = False

    while not out_of_bounds:
        grid.mark_visited(pos, direction)
        pos, direction, out_of_bounds = grid.next_state(pos, direction)

    return grid.count_visited()


grid_txt = open("inputs/6.txt").read()
distinct_positions(grid_txt)  # 5269

5269

#### 6.2

In [18]:
def looping_obstacles(grid_txt):
    n_looping_obstacles = 0
    grid = Grid(grid_txt)
    initial_pos = np.array(grid.find_guard_position())
    initial_direction = get_direction(grid.get(initial_pos))

    for i in range(grid.n):
        for j in range(grid.m):
            if grid.grid[i, j] != ".":
                continue

            grid.reset_history()
            grid.grid[i, j] = "#"
            pos = initial_pos
            direction = initial_direction
            out_of_bounds = False
            while not out_of_bounds:
                if grid.already_visited(pos, direction):
                    n_looping_obstacles += 1
                    break
                grid.mark_visited(pos, direction)
                pos, direction, out_of_bounds = grid.next_state(pos, direction)

            grid.grid[i, j] = "."

    return n_looping_obstacles


looping_obstacles(grid_txt)  # 1957


KeyboardInterrupt: 

#### 7.1

In [19]:
def get_equations(equations_txt):
    return [
        (int(test), np.fromstring(numbers, int, sep=" "))
        for test, numbers in (equation.split(": ") for equation in equations_txt.splitlines())
    ]

In [20]:
def check_combinations(test, numbers, current_value=0):
    if numbers.size == 0:
        return test == current_value
    if current_value > test:
        return False
    return check_combinations(test, numbers[1:], current_value=current_value + numbers[0]) or check_combinations(
        test, numbers[1:], current_value=current_value * numbers[0]
    )


equations_txt = open("inputs/7.txt").read()
equations = get_equations(equations_txt)

sum(test for test, numbers in equations if check_combinations(test, numbers))

20281182715321

#### 7.2

In [21]:
def check_combinations_with_concat(test, numbers, current_value=0):
    if numbers.size == 0:
        return test == current_value
    if current_value > test:
        return False
    return (
        check_combinations_with_concat(test, numbers[1:], current_value=current_value + numbers[0])
        or check_combinations_with_concat(test, numbers[1:], current_value=current_value * numbers[0])
        or check_combinations_with_concat(test, numbers[1:], current_value=int(str(current_value) + str(numbers[0])))
    )

In [22]:
sum(test for test, numbers in equations if check_combinations_with_concat(test, numbers))

159490400628354

#### 8.1

In [23]:
def get_antenna_locs(antenna_map):
    locs = defaultdict(list)
    for i, line in enumerate(antenna_map):
        for j, symbol in enumerate(line):
            if symbol != ".":
                locs[symbol].append(np.array([i, j]))
    return locs


antenna_map_txt = open("inputs/8.txt").read()

antenna_map_example_txt = """\
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............"""

In [24]:
def within_bounds(coords, max_coords):
    return (coords >= 0).all() and (coords < max_coords).all()


def get_n_antinodes(antenna_map_txt):
    antenna_map = antenna_map_txt.splitlines()
    antenna_locs = get_antenna_locs(antenna_map)

    antinode_locs = set()
    max_coords = np.array([len(antenna_map), len(antenna_map[0])])
    for freq_locs in antenna_locs.values():
        for c1, c2 in combinations(freq_locs, 2):
            diff = c2 - c1
            for antinode in (c1 - diff, c2 + diff):
                if within_bounds(antinode, max_coords):
                    antinode_locs.add(tuple(antinode))

    return len(antinode_locs)


get_n_antinodes(antenna_map_txt)

344

#### 8.2

In [25]:
def get_n_resonant_antinodes(antenna_map_txt):
    antenna_map = antenna_map_txt.splitlines()
    antenna_locs = get_antenna_locs(antenna_map)

    antinode_locs = set()
    max_coords = np.array([len(antenna_map), len(antenna_map[0])])

    for freq_locs in antenna_locs.values():
        for c1, c2 in combinations(freq_locs, 2):
            for pos, offset in ((c1.copy(), c1 - c2), (c2.copy(), c2 - c1)):
                while within_bounds(pos, max_coords):
                    antinode_locs.add(tuple(pos))
                    pos += offset
    return len(antinode_locs)


get_n_resonant_antinodes(antenna_map_txt)

1182

#### 9.1

In [26]:
disk_map_txt = open("inputs/9.txt").read()[:-1]


def position_sum(curr_pos, new_pos):
    return (new_pos + curr_pos - 1) * (new_pos - curr_pos) // 2


def update_checksum_and_position(checksum, curr_pos, digit, digit_idx):
    new_pos = curr_pos + digit
    checksum += digit_idx * position_sum(curr_pos, new_pos)
    return checksum, new_pos


def reordered_checksum(disk_map_txt):
    digits = list(map(int, disk_map_txt[::2]))
    spaces = list(map(int, disk_map_txt[1::2]))
    checksum = 0
    curr_pos = 0
    end_idx = len(digits) - 1
    for idx, _ in enumerate(digits):
        checksum, curr_pos = update_checksum_and_position(checksum, curr_pos, digits[idx], idx)

        if end_idx <= idx:
            break

        while end_idx > idx and spaces[idx] > 0:
            filled_positions = min(digits[end_idx], spaces[idx])
            spaces[idx] -= filled_positions
            digits[end_idx] -= filled_positions
            checksum, curr_pos = update_checksum_and_position(checksum, curr_pos, filled_positions, end_idx)
            if digits[end_idx] == 0:
                end_idx -= 1

    return checksum


disk_map_example_txt = "2333133121414131402"
reordered_checksum(disk_map_example_txt)

1928

#### 9.2

In [27]:
def defragmented_checksum(disk_map_txt):
    digits_per_pos = [[(dig_id, int(symbol))] for dig_id, symbol in enumerate(disk_map_txt[::2])]
    spaces = list(map(int, disk_map_txt[1::2])) + [0]

    for end_idx, _ in reversed(list(enumerate(digits_per_pos))):
        dig_id, digit = digits_per_pos[end_idx][0]
        for space_idx, _ in enumerate(spaces):
            if space_idx >= end_idx:
                break
            if spaces[space_idx] >= digit:
                spaces[space_idx] -= digit
                digits_per_pos[space_idx].append((dig_id, digit))
                spaces[dig_id - 1] += digit
                digits_per_pos[dig_id][0] = (dig_id, 0)
                break

    curr_pos = 0
    checksum = 0
    for idx, digit_list in enumerate(digits_per_pos):
        for dig_id, digit in digit_list:
            checksum, curr_pos = update_checksum_and_position(checksum, curr_pos, digit, dig_id)
        curr_pos += spaces[idx]

    return checksum


defragmented_checksum(disk_map_txt)

6408966547049

#### 10.1

In [28]:
topo_map_txt = open("inputs/10.txt").read()

In [29]:
def map_score(topo_map_txt):
    topo_map = np.array([list(map(int, line)) for line in topo_map_txt.splitlines()])

    def out_of_bounds(i, j):
        return i < 0 or j < 0 or i >= topo_map.shape[0] or j >= topo_map.shape[1]

    deltas = np.array(
        [
            [0, 1],
            [1, 0],
            [0, -1],
            [-1, 0],
        ]
    )
    score = 0
    trailheads = np.argwhere(topo_map == 0)
    for trailhead in trailheads:
        visited = np.zeros_like(topo_map, dtype=bool)
        stack = [trailhead]
        while stack:
            top = stack.pop()
            if visited[tuple(top)]:
                continue

            if topo_map[tuple(top)] == 9:
                score += 1
            visited[tuple(top)] = True
            for delta in deltas:
                new_pos = top + delta
                if (
                    not out_of_bounds(*new_pos)
                    and not visited[tuple(new_pos)]
                    and topo_map[tuple(new_pos)] == topo_map[tuple(top)] + 1
                ):
                    stack.append(new_pos)
    return score


map_score(topo_map_txt)

566

#### 10.2

In [30]:
def map_score(topo_map_txt):
    topo_map = np.array([list(map(int, line)) for line in topo_map_txt.splitlines()])

    def out_of_bounds(i, j):
        return i < 0 or j < 0 or i >= topo_map.shape[0] or j >= topo_map.shape[1]

    deltas = np.array(
        [
            [0, 1],
            [1, 0],
            [0, -1],
            [-1, 0],
        ]
    )
    score = 0
    trailheads = np.argwhere(topo_map == 0)

    visited = np.zeros_like(topo_map, dtype=int)
    for pos in trailheads:
        visited[tuple(pos)] = 1
    new_stack = list(trailheads)
    height = 0

    while new_stack:
        stack = new_stack
        new_stack = []
        for pos in stack:
            for delta in deltas:
                new_pos = pos + delta
                if not out_of_bounds(*new_pos) and topo_map[tuple(new_pos)] == height + 1:
                    if not visited[tuple(new_pos)]:
                        new_stack.append(new_pos)
                    visited[tuple(new_pos)] += visited[tuple(pos)]
        height += 1

    score = sum(visited[tuple(pos)] for pos in np.argwhere(topo_map == 9))
    return score


map_score(topo_map_txt)

np.int64(1324)

#### 11.1

In [31]:
def evolve(number):
    if number == 0:
        return [1]

    digits = str(number)
    n_digits = len(digits)
    if n_digits % 2 == 0:
        return [int(digits[: n_digits // 2]), int(digits[n_digits // 2 :])]

    return [number * 2024]


def get_n_stones(stones_txt, n_blinks=25):
    stone_counts = dict(Counter(map(int, stones_txt.split())))

    for i in range(n_blinks):
        next_counts = defaultdict(int)
        for number, count in stone_counts.items():
            for evolved_number in evolve(number):
                next_counts[evolved_number] += count
        stone_counts = next_counts
    return sum(stone_counts.values())


stones_txt = open("inputs/11.txt").read()

get_n_stones(stones_txt, n_blinks=25)

190865

#### 11.2

In [32]:
get_n_stones(stones_txt, n_blinks=75)

225404711855335

#### 12.1

In [33]:
def add_delta(coords, delta):
    return (coords[0] + delta[0], coords[1] + delta[1])


def fencing_price(garden_txt):
    garden = np.array(list(map(list, garden_txt.split())))

    visited = np.zeros_like(garden, dtype=bool)
    deltas = (
        (-1, 0),
        (0, 1),
        (1, 0),
        (0, -1),
    )

    def is_connected(nb_coords, plant_type):
        return not out_of_bounds(*nb_coords) and garden[nb_coords] == plant_type

    def out_of_bounds(i, j):
        return i < 0 or j < 0 or i >= garden.shape[0] or j >= garden.shape[1]

    def dfs(coords, plant_type, component):
        component.append(coords)
        visited[coords] = True
        perimeter = 4
        for delta in deltas:
            nb_coords = add_delta(coords, delta)
            if is_connected(nb_coords, plant_type):
                perimeter -= 1
                if not visited[nb_coords]:
                    perimeter += dfs(nb_coords, plant_type, component)
        return perimeter

    price = 0
    for coords, curr_plot in np.ndenumerate(garden):
        if not visited[coords]:
            connected_component = []
            perimeter = dfs(coords, curr_plot, connected_component)
            price += len(connected_component) * perimeter
    return price

In [34]:
garden_txt = open("inputs/12.txt").read()

fencing_price(garden_txt)

1464678

#### 12.2

In [35]:
def discounted_fencing_price(garden_txt):
    garden = np.array(list(map(list, garden_txt.split())))

    visited = np.zeros_like(garden, dtype=bool)
    deltas = (
        (-1, 0),
        (0, 1),
        (1, 0),
        (0, -1),
    )

    def is_connected(nb_coords, plant_type):
        return not out_of_bounds(*nb_coords) and garden[nb_coords] == plant_type

    def out_of_bounds(i, j):
        return i < 0 or j < 0 or i >= garden.shape[0] or j >= garden.shape[1]

    def dfs(coords, plant_type, component):
        component.append(coords)
        visited[coords] = True

        for delta in deltas:
            nb_coords = add_delta(coords, delta)
            if is_connected(nb_coords, plant_type):
                if not visited[nb_coords]:
                    dfs(nb_coords, plant_type, component)

    price = 0
    for coords, curr_plot in np.ndenumerate(garden):
        if not visited[coords]:
            connected_component = []
            dfs(coords, curr_plot, connected_component)
            sides = 0
            for i, coords in enumerate(connected_component):
                for direction in Direction:
                    delta = deltas[direction.value]
                    if not is_connected(add_delta(coords, delta), curr_plot):
                        left_rotated_nb_coords = add_delta(coords, deltas[rotate_left(direction).value])
                        if not is_connected(left_rotated_nb_coords, curr_plot) or is_connected(
                            add_delta(left_rotated_nb_coords, delta), curr_plot
                        ):
                            sides += 1

            price += len(connected_component) * sides
    return price


discounted_fencing_price(garden_txt)

877492

#### 13.1

In [13]:
def process_machine_input(line):
    return re.findall(r"Button A: X\+([0-9]+), Y\+([0-9]+)\nButton B: X\+([0-9]+), Y\+([0-9]+)\nPrize: X=([0-9]+), Y=([0-9]+)", line)[
        0
    ]


def min_tokens(machine_input_line, fix_unit_conversion=False):
    x_a, y_a, x_b, y_b, price_x, price_y = map(int, process_machine_input(machine_input_line))
    A = np.array(
        [
            [x_a, x_b],
            [y_a, y_b],
        ]
    )

    assert np.linalg.det(A) != 0

    p = np.array([price_x, price_y])
    if fix_unit_conversion:
        p += 10000000000000

    n_tokens = np.linalg.solve(A, p).round().astype(int)

    if not np.array_equal(A @ n_tokens, p):
        return 0

    return 3 * n_tokens[0] + n_tokens[1]


machine_inputs_txt = open("inputs/13.txt").read()[:-1].split("\n\n")
sum(map(min_tokens, machine_inputs_txt))


np.int64(37686)

#### 13.2

In [11]:
sum(min_tokens(line, fix_unit_conversion=True) for line in machine_inputs_txt)

np.int64(77204516023437)

#### 14.1

In [18]:
robot_inputs_txt = open("inputs/14.txt").read()

In [19]:
def safety_factor(robot_inputs_txt, bathroom_shape, n_seconds=100):
    bathroom_shape = np.array(bathroom_shape)
    positions, velocities = np.split(
        np.array(re.findall(r"p=([0-9]+),([0-9]+) v=(-?[0-9]+),(-?[0-9]+)", robot_inputs_txt), dtype=int),
        2,
        axis=1,
    )

    new_positions = (positions + velocities * n_seconds) % bathroom_shape
    split_lines = bathroom_shape // 2

    ans = 1
    for in_quadrant in [
        (new_positions[:, 0] < split_lines[0]) & (new_positions[:, 1] < split_lines[1]),
        (new_positions[:, 0] < split_lines[0]) & (new_positions[:, 1] > split_lines[1]),
        (new_positions[:, 0] > split_lines[0]) & (new_positions[:, 1] < split_lines[1]),
        (new_positions[:, 0] > split_lines[0]) & (new_positions[:, 1] > split_lines[1]),
    ]:
        ans *= np.count_nonzero(in_quadrant)
    return ans


safety_factor(robot_inputs_txt, (101, 103))

209409792

In [67]:
def largest_component(positions, bathroom_shape):
    grid = np.zeros(bathroom_shape, int)
    np.add.at(grid, (positions[:, 0], positions[:, 1]), 1)

    assert np.sum(grid) == len(positions)
    visited = np.zeros(grid.shape, bool)

    def dfs(coords, n_robots):
        if visited[coords]:
            return n_robots
        visited[coords] = True
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue

                x_next = coords[0] + dx
                y_next = coords[1] + dy
                if x_next < 0 or y_next < 0 or x_next >= bathroom_shape[0] or y_next >= bathroom_shape[1]:
                    continue
                if grid[x_next, y_next] > 0:
                    n_robots = dfs((x_next, y_next), n_robots)
        return n_robots + 1

    return max(dfs((x, y), 0) for x, y in positions)


def christmas_tree_time(robot_inputs_txt, bathroom_shape, t_max=100, verbose=False):
    bathroom_shape = np.array(bathroom_shape)
    positions, velocities = np.split(
        np.array(re.findall(r"p=([0-9]+),([0-9]+) v=(-?[0-9]+),(-?[0-9]+)", robot_inputs_txt), dtype=int),
        2,
        axis=1,
    )

    best_so_far = 1
    for t in range(1, t_max + 1):
        positions = (positions + velocities) % bathroom_shape
        conn = largest_component(positions, bathroom_shape)
        if conn > best_so_far:
            if conn > positions.shape[0] * 0.1:
                if verbose:
                    grid = np.zeros(bathroom_shape, int)
                    np.add.at(grid, (positions[:, 0], positions[:, 1]), 1)
                    for line in grid.T:
                        print("".join([str(val) if val > 0 else " " for val in line]))
                return t
            best_so_far = conn
    return -1


christmas_tree_time(robot_inputs_txt, (101, 103), t_max=100000)

8006

#### 15.1